In [1]:
!pwd

/content


In [2]:
!git clone https://github.com/EMBL-EBI-ABC/PerturbationCatalogue.git

fatal: destination path 'PerturbationCatalogue' already exists and is not an empty directory.


In [3]:
%cd PerturbationCatalogue/data_exploration/

/content/PerturbationCatalogue/data_exploration


In [4]:
!git pull
!git switch aleks/curating_orion_2025

Already up to date.
Already on 'aleks/curating_orion_2025'
Your branch is up to date with 'origin/aleks/curating_orion_2025'.


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 188.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 167.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.1
    Uninstalling numpy-2.3.1:
      Successfully uninstalled numpy-2.3.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.3.1
    Uninstalling pandas-2.3.1:
      Successfully uninstalled pandas-2.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.1 which is incompatible.
mizani 0.13.5 requires pandas>=2.2.0, but you have pandas 2.1.1 which is incompatible.
thinc 8.3.6 requires num

In [6]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you want to use for this configuration.
To use a federated user account, exit this command and sign in to the gcloud CLI
 with your login configuration file, then run this command again.

Select an account:
 [1] zakirov@ebi.ac.uk
 [2] Sign in with a new Google Account
 [3] Skip this step
Please enter your numeric choice:  1

You are signed in as: [zakirov@ebi.ac.uk].

This account has a lot of projects! Listing them all can take a while.
 [1] Enter a project ID
 [2] Create a new project
 [3] List projects
Please enter your numeric choice:  1

Enter project ID you would like to use:  prj-ext-dev-pertcat

In [7]:
!mkdir -p Perturbseq/non_curated/h5ad/
!gsutil -m cp gs://perturbation-catalogue-lake/perturbseq/non_curated/orion_2025_hek293t.h5ad Perturbseq/non_curated/h5ad/

Copying gs://perturbation-catalogue-lake/perturbseq/non_curated/orion_2025_hek293t.h5ad...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

-
Operation completed over 1 objects/326.1 GiB.                                    


# Import

In [5]:
import pandas as pd

import sys

sys.path.append("../../")
from curation_tools.curation_tools import CuratedDataset
from curation_tools.perturbseq_anndata_schema import ObsSchema, VarSchema

from curation_tools.unified_metadata_schema.unified_metadata_schema import Experiment

/usr/local/lib/python3.11/dist-packages/pandera/_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


# Pre-process the data

### Load the data and construct the dummy anndata object to explore the dataset offline

In [5]:
from scipy.sparse import csr_matrix
import numpy as np
import scanpy as sc
import snapatac2 as snap

backed_data = snap.read('Perturbseq/non_curated/h5ad/orion_2025_hek293t.h5ad')

obs = backed_data.obs[:].to_pandas()
var = backed_data.var[:].to_pandas()

N1 = obs.shape[0]  # Number of objects (e.g., cells)
N2 = var.shape[0]  # Number of categories (e.g., genes)
rows = np.arange(N1)
cols = (np.floor(np.random.permutation(N1)/float(N1)*N2)).astype(int) # Randomly pick N1 objects and assign to N2 categories in almost equal proportion
w = np.ones(N1)
conn_matrix = csr_matrix((w, (rows, cols)), shape=(N1, N2), dtype=int)

adata = sc.AnnData(X=conn_matrix, obs=obs, var=var)

# save the non-curated AnnData object
adata.write_h5ad("Perturbseq/non_curated/h5ad/orion_2025_hek293t_dummy.h5ad")

/tmp/ipython-input-5-3476015825.py:8: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  obs = backed_data.obs[:].to_pandas()


ValueError: axis 1 index 0 exceeds matrix dimension 0

In [8]:
backed_data.close()

# Initialise the dataset object

In [6]:
cur_data = CuratedDataset(
    obs_schema=ObsSchema,
    var_schema=VarSchema,
    exp_metadata_schema=Experiment,
    noncurated_path = "Perturbseq/non_curated/h5ad/orion_2025_hek293t.h5ad"
)

# Load the dataset

In [7]:
# cur_data.download_data()
cur_data.load_data()
# show the data
cur_data.show_obs()

Loading data from Perturbseq/non_curated/h5ad/orion_2025_hek293t.h5ad
Observation data:
DataFrame shape: (4534299, 9)
--------------------------------------------------
                                             sample  num_features  \
AAACCAAAGAAGTAGC-HEK293T_Batch1      HEK293T_Batch1             2   
AAACCAAAGAGTACGG-HEK293T_Batch1      HEK293T_Batch1             2   
AAACCAAAGAGTGGGC-HEK293T_Batch1      HEK293T_Batch1             2   
AAACCAAAGAGTGTAA-HEK293T_Batch1      HEK293T_Batch1             2   
AAACCAAAGATACGGG-HEK293T_Batch1      HEK293T_Batch1             2   
...                                             ...           ...   
GTTGTCCGTGGGATGG-HEK293T_Batch223  HEK293T_Batch223             2   
GTTGTCTTCAACAGCC-HEK293T_Batch223  HEK293T_Batch223             2   
GTTGTCTTCAGTACTG-HEK293T_Batch223  HEK293T_Batch223             2   
GTTGTCTTCCGCCGTT-HEK293T_Batch223  HEK293T_Batch223             2   
GTTGTGCAGGCTTGCG-HEK293T_Batch223  HEK293T_Batch223             2   

  

In [9]:
cur_data.adata.var['gene_symbol'] = cur_data.adata.var.index

In [10]:
cur_data.show_var()

Variable data:
DataFrame shape: (38606, 1)
--------------------------------------------------
                     gene_symbol
DDX11L2                  DDX11L2
MIR1302-2HG          MIR1302-2HG
FAM138A                  FAM138A
ENSG00000290826  ENSG00000290826
OR4F5                      OR4F5
...                          ...
ENSG00000277836  ENSG00000277836
ENSG00000278633  ENSG00000278633
ENSG00000276017  ENSG00000276017
ENSG00000278817  ENSG00000278817
ENSG00000277196  ENSG00000277196

[38606 rows x 1 columns]
--------------------------------------------------


# OBS slot curation

In [14]:
cur_data.adata.obs.isna().sum()

,0
sample,0
num_features,0
guide_target,0
gene_target,0
n_genes_by_counts,0
total_counts,0
total_counts_mt,0
pct_counts_mt,0
pass_guide_filter,0


### Extract perturbation symbols

### Rename `guide_target` to `perturbation_name`

In [15]:
cur_data.rename_columns(slot = 'obs', name_dict = {'guide_target': 'perturbation_name'})

Renamed columns in adata.obs: {'guide_target': 'perturbation_name'}


### Rename `gene_target` to `perturbed_target_symbol`

In [16]:
cur_data.rename_columns(slot = 'obs', name_dict = {'gene_target': 'perturbed_target_symbol'})

Renamed columns in adata.obs: {'gene_target': 'perturbed_target_symbol'}


#### Clean up `perturbed_target_symbol` column

In [17]:
cur_data.replace_entries(
    slot="obs",
    column="perturbed_target_symbol",
    map_dict={
        "Non-Targeting": "control_nontargeting"
    }
)

cur_data.show_unique(slot = 'obs', column = 'perturbed_target_symbol')

Streaming output truncated to the last 5000 lines.
 'RPA4',
 'RPAIN',
 'RPAP1',
 'RPAP2',
 'RPAP3',
 'RPE',
 'RPE65',
 'RPF1',
 'RPF2',
 'RPGR',
 'RPGRIP1',
 'RPGRIP1L',
 'RPH3A',
 'RPH3AL',
 'RPIA',
 'RPL10',
 'RPL10A',
 'RPL10L',
 'RPL11',
 'RPL12',
 'RPL13',
 'RPL13A',
 'RPL14',
 'RPL15',
 'RPL17',
 'RPL18',
 'RPL18A',
 'RPL19',
 'RPL21',
 'RPL22',
 'RPL22L1',
 'RPL23',
 'RPL23A',
 'RPL24',
 'RPL26',
 'RPL26L1',
 'RPL27',
 'RPL27A',
 'RPL28',
 'RPL29',
 'RPL3',
 'RPL30',
 'RPL31',
 'RPL32',
 'RPL34',
 'RPL35',
 'RPL35A',
 'RPL36',
 'RPL36A',
 'RPL36AL',
 'RPL37',
 'RPL37A',
 'RPL38',
 'RPL39',
 'RPL39L',
 'RPL3L',
 'RPL4',
 'RPL41',
 'RPL5',
 'RPL6',
 'RPL7',
 'RPL7A',
 'RPL7L1',
 'RPL8',
 'RPL9',
 'RPLP0',
 'RPLP1',
 'RPLP2',
 'RPN1',
 'RPN2',
 'RPP14',
 'RPP21',
 'RPP25',
 'RPP25L',
 'RPP30',
 'RPP40',
 'RPRD1A',
 'RPRD1B',
 'RPRD2',
 'RPRM',
 'RPRML',
 'RPS10',
 'RPS11',
 'RPS12',
 'RPS13',
 'RPS14',
 'RPS15',
 'RPS15A',
 'RPS16',
 'RPS17',
 'RPS18',
 'RPS19',
 'RPS19BP1',
 'RPS2

### Standardise perturbation targets

In [18]:
cur_data.standardize_genes(
    slot='obs',
    input_column='perturbed_target_symbol',
    input_column_type='gene_symbol',
    multiple_entries=False
)

/content/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:1271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_df["synonyms"] = map_df["synonyms"].str.split("|")


Mapped potential synonyms in perturbed_target_symbol of the provided dataframe to gene symbols
Converted 17823/17833 gene symbols/ENSG IDs to standardized gene symbols/ENSG IDs
--------------------------------------------------


### Add `perturbed_target_number` column

In [19]:
cur_data.count_entries(
    slot='obs',
    input_column='perturbed_target_symbol',
    count_column_name='perturbed_target_number',
    sep='|'
)

Counted entries in column perturbed_target_symbol of adata.obs and stored in perturbed_target_number


In [20]:
cur_data.adata.obs.head()

,n_genes_by_counts,total_counts,pass_guide_filter,perturbation_name,num_features,sample,pct_counts_mt,total_counts_mt,perturbed_target_ensg,perturbed_target_symbol,perturbed_target_biotype,perturbed_target_coord,perturbed_target_number
index,,,,,,,,,,,,,
AAACCAAAGAAGTAGC-HEK293T_Batch1,6856,24243.0,True,FAM180A_P1P2-1|FAM180A_P1P2-2,2,HEK293T_Batch1,2.668812,647.0,ENSG00000189320,FAM180A,protein_coding,chr7:135728348-135748813;-1,1
AAACCAAAGAGTACGG-HEK293T_Batch1,6636,20847.0,True,non-targeting_03016|non-targeting_03214,2,HEK293T_Batch1,4.250012,886.0,control_nontargeting,control_nontargeting,NaN,NaN,1
AAACCAAAGAGTGGGC-HEK293T_Batch1,5353,14035.0,True,PCDHGC4_P1P2-1|PCDHGC4_P1P2-2,2,HEK293T_Batch1,4.346277,610.0,ENSG00000242419,PCDHGC4,protein_coding,chr5:141484997-141512975;1,1
AAACCAAAGAGTGTAA-HEK293T_Batch1,6358,20939.0,True,USP30_P1P2-1|USP30_P1P2-2,2,HEK293T_Batch1,2.812933,589.0,ENSG00000135093,USP30,protein_coding,chr12:109023089-109088023;1,1
AAACCAAAGATACGGG-HEK293T_Batch1,5527,14083.0,True,KLRD1_P1P2-1|KLRD1_P1P2-2,2,HEK293T_Batch1,3.379961,476.0,ENSG00000134539,KLRD1,protein_coding,chr12:10226058-10329608;1,1


### Add guide RNA information

The original pre-print does not contain information about guide RNAs in the `obs` slot.

However, the pre-print references the source of the library - [Maximizing CRISPRi efficacy and accessibility with dual-sgRNA libraries and optimal effectors. Replogle et al 2022](https://doi.org/10.7554/eLife.81856).

Supplementary table s4 contains the information about guide RNAs used in the experiment:

Supplementary table S4: https://elifesciences.org/download/aHR0cHM6Ly9jZG4uZWxpZmVzY2llbmNlcy5vcmcvYXJ0aWNsZXMvODE4NTYvZWxpZmUtODE4NTYtc3VwcDQtdjIueGxzeA--/elife-81856-supp4-v2.xlsx?_hash=AmEXp%2BOl1z1Y4JCBAaVhST5ImQmKAGdfEjGhuJbSJ9Q%3D

In [21]:
# get synonyms from the gene ontology
syn_df = cur_data.gene_ont[["synonyms", "symbol"]].drop_duplicates()
syn_df["symbol"] = syn_df["symbol"].fillna("")
syn_df["synonyms"] = syn_df["synonyms"].fillna("")
syn_df["all_genes"] = (
    (syn_df["symbol"] + "|" + syn_df["synonyms"]).str.lstrip("|").str.rstrip("|")
)
syn_df["all_genes_match"] = syn_df["all_genes"].str.split("|")
syn_df = syn_df.explode("all_genes_match")
syn_df["all_genes_match"] = syn_df["all_genes_match"].str.upper()

# read data from supplementary files
guide_ids_df = pd.read_csv(
    "Perturbseq/supplementary/orion_2025_weissman_dual_crispri_sgrnas.csv"
)
guide_ids_df["gene"] = guide_ids_df["gene"].str.upper()

guide_ids_df = guide_ids_df.merge(
    syn_df, left_on="gene", right_on="all_genes_match", how="left"
)

guide_ids_df["all_genes"] = guide_ids_df["all_genes"].str.split("|")
guide_ids_df = guide_ids_df.explode("all_genes").drop_duplicates()

# create a perturbation name column
guide_ids_df["perturbation_name"] = (
    guide_ids_df["all_genes"]
    + "_"
    + guide_ids_df["transcript"]
    + "|"
    + guide_ids_df["all_genes"]
    + "_"
    + guide_ids_df["transcript"]
)

guide_ids_df["perturbation_name"] = guide_ids_df["perturbation_name"].str.upper()

# replace commas with underscores in the perturbation name
guide_ids_df["perturbation_name"] = guide_ids_df["perturbation_name"].str.replace(
    ",", "_"
)

# create a dual guide RNA sequence column
guide_ids_df["dual_guide_sequence"] = (
    guide_ids_df["protospacer_A"] + "|" + guide_ids_df["protospacer_B"]
).str.upper()

guide_ids_df.loc[
    guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "perturbation_name"
] = (
    guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "gene"
    ]
    + "_"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "transcript"
    ]
    + "|"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "gene"
    ]
    + "_"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "transcript"
    ]
)

# replace NEGATIVE_CONTROL with actual non targeting guide names
guide_ids_df.loc[guide_ids_df["gene"] == "NEGATIVE_CONTROL", "perturbation_name"] = (
    guide_ids_df.loc[guide_ids_df["gene"] == "NEGATIVE_CONTROL", "sgID_AB"].str.upper()
)

# create dictionaries for guide sequences mapping
guide_ids_dict = (
    guide_ids_df[["perturbation_name", "dual_guide_sequence"]]
    .dropna()
    .drop_duplicates()
    .set_index("perturbation_name")["dual_guide_sequence"]
    .to_dict()
)

# add guide sequences to the obs dataframe
obj_df = cur_data.adata.obs.copy()

# remove the numeric suffices from the perturbation names
obj_df["perturbation_name_matching"] = obj_df["perturbation_name"].str.replace(
    r"-\d(?=\||$)", "", regex=True
)
obj_df["perturbation_name_matching"] = obj_df["perturbation_name_matching"].str.upper()

# map the guide sequences to the perturbation names
obj_df["guide_sequence"] = obj_df["perturbation_name_matching"].map(guide_ids_dict)

obj_df = obj_df[["guide_sequence"]]

obj_df

,guide_sequence
index,
AAACCAAAGAAGTAGC-HEK293T_Batch1,GGAGAGCGTCAAATGAAGAG|GGAGGCTGAAGGCTGCGTCC
AAACCAAAGAGTACGG-HEK293T_Batch1,GGAGGCTATCTGCTAGACTT|GTCCACATGCGGATAGACAC
AAACCAAAGAGTGGGC-HEK293T_Batch1,GGGCCGAACCGCGCCAGAGC|GCGACCGAGAGGCACTGTGT
AAACCAAAGAGTGTAA-HEK293T_Batch1,GTGCCGAAGAGGCCGGGACC|GACGACGGTTCCCGAGACAG
AAACCAAAGATACGGG-HEK293T_Batch1,GATGTTCTTGAATCATCTAT|GGGTTCCTGGAACACTTCAG
...,...
GTTGTCCGTGGGATGG-HEK293T_Batch223,GGGGGAATCTGCAAGACCAT|GAGAAGGAGCGAGGCTAAAG
GTTGTCTTCAACAGCC-HEK293T_Batch223,GGAGCACAGGCGCTTCACCG|GGGCAGTCTCGGCAGAGCAC
GTTGTCTTCAGTACTG-HEK293T_Batch223,GAAGTTAGGTGTCCAGAGCA|GAGGTGTCCAGAGCACGGTA


In [22]:
# merge guide sequences with main
cur_data.adata.obs = cur_data.adata.obs.merge(
    obj_df,
    left_index=True,
    right_index=True,
    how="left",
)

cur_data.adata.obs

,n_genes_by_counts,total_counts,pass_guide_filter,perturbation_name,num_features,sample,pct_counts_mt,total_counts_mt,perturbed_target_ensg,perturbed_target_symbol,perturbed_target_biotype,perturbed_target_coord,perturbed_target_number,guide_sequence
index,,,,,,,,,,,,,,
AAACCAAAGAAGTAGC-HEK293T_Batch1,6856,24243.0,True,FAM180A_P1P2-1|FAM180A_P1P2-2,2,HEK293T_Batch1,2.668812,647.0,ENSG00000189320,FAM180A,protein_coding,chr7:135728348-135748813;-1,1,GGAGAGCGTCAAATGAAGAG|GGAGGCTGAAGGCTGCGTCC
AAACCAAAGAGTACGG-HEK293T_Batch1,6636,20847.0,True,non-targeting_03016|non-targeting_03214,2,HEK293T_Batch1,4.250012,886.0,control_nontargeting,control_nontargeting,NaN,NaN,1,GGAGGCTATCTGCTAGACTT|GTCCACATGCGGATAGACAC
AAACCAAAGAGTGGGC-HEK293T_Batch1,5353,14035.0,True,PCDHGC4_P1P2-1|PCDHGC4_P1P2-2,2,HEK293T_Batch1,4.346277,610.0,ENSG00000242419,PCDHGC4,protein_coding,chr5:141484997-141512975;1,1,GGGCCGAACCGCGCCAGAGC|GCGACCGAGAGGCACTGTGT
AAACCAAAGAGTGTAA-HEK293T_Batch1,6358,20939.0,True,USP30_P1P2-1|USP30_P1P2-2,2,HEK293T_Batch1,2.812933,589.0,ENSG00000135093,USP30,protein_coding,chr12:109023089-109088023;1,1,GTGCCGAAGAGGCCGGGACC|GACGACGGTTCCCGAGACAG
AAACCAAAGATACGGG-HEK293T_Batch1,5527,14083.0,True,KLRD1_P1P2-1|KLRD1_P1P2-2,2,HEK293T_Batch1,3.379961,476.0,ENSG00000134539,KLRD1,protein_coding,chr12:10226058-10329608;1,1,GATGTTCTTGAATCATCTAT|GGGTTCCTGGAACACTTCAG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTGTCCGTGGGATGG-HEK293T_Batch223,8112,29296.0,True,non-targeting_00183|non-targeting_01885,2,HEK293T_Batch223,1.781813,522.0,control_nontargeting,control_nontargeting,NaN,NaN,1,GGGGGAATCTGCAAGACCAT|GAGAAGGAGCGAGGCTAAAG
GTTGTCTTCAACAGCC-HEK293T_Batch223,10568,88551.0,True,AKAP13_P1P2-1|AKAP13_P1P2-2,2,HEK293T_Batch223,3.427403,3035.0,ENSG00000170776,AKAP13,protein_coding,chr15:85380571-85749358;1,1,GGAGCACAGGCGCTTCACCG|GGGCAGTCTCGGCAGAGCAC
GTTGTCTTCAGTACTG-HEK293T_Batch223,10735,81286.0,True,TTF2_P1P2-1|TTF2_P1P2-2,2,HEK293T_Batch223,2.224245,1808.0,ENSG00000116830,TTF2,protein_coding,chr1:117060326-117107453;1,1,GAAGTTAGGTGTCCAGAGCA|GAGGTGTCCAGAGCACGGTA


### Add treatment information

Add treatment information with the dataset

In [23]:
cur_data.create_columns(
    slot="obs",
    col_dict={
        "treatment_label": None,
        "treatment_id": None
    }
)

Column treatment_label added to adata.obs
Column treatment_id added to adata.obs


### Add perturbation information

In [24]:
cur_data.create_columns(
    slot="obs",
    col_dict={
        "perturbation_type_label": "CRISPRi",
        "perturbation_type_id": None
    }
)

Column perturbation_type_label added to adata.obs
Column perturbation_type_id added to adata.obs


### Add timepoint information

In [25]:
cur_data.create_columns(
    slot="obs",
    col_dict={"timepoint": "P0DT0H0M0S"},
)

Column timepoint added to adata.obs


### Add model system information

In [26]:
cur_data.create_columns(
    slot="obs",
    col_dict={
        "model_system_label": "cell line",
        "model_system_id": None
    }
)

Column model_system_label added to adata.obs
Column model_system_id added to adata.obs


### Add tissue information

In [27]:
cur_data.create_columns(
    slot='obs',
    col_dict={
        'tissue': 'kidney'
    }
)

cur_data.standardize_ontology(
    input_column='tissue',
    column_type='term_name',
    ontology_type='tissue',
    overwrite=True
)

Column tissue added to adata.obs
Mapped 1 tissue ontology terms from `tissue` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
  input_column input_column_lower name_lower     ontology_id
0       kidney             kidney     kidney  UBERON:0002113
--------------------------------------------------


### Add cell type information

In [28]:
cur_data.create_columns(
    slot='obs',
    col_dict={
        'celltype': 'kidney epithelial cell'
    }
)

cur_data.standardize_ontology(
    input_column='celltype',
    column_type='term_name',
    ontology_type='cell_type'
)

Column celltype added to adata.obs
Mapped 1 cell_type ontology terms from `celltype` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
             input_column      input_column_lower              name_lower  \
0  kidney epithelial cell  kidney epithelial cell  kidney epithelial cell   

  ontology_id  
0  CL:0002518  
--------------------------------------------------


### Add cell line information

In [29]:
cur_data.create_columns(
    slot='obs',
    col_dict={
        'cell_line': 'HEK293T cell'
    }
)

cur_data.standardize_ontology(
    input_column='cell_line',
    column_type='term_name',
    ontology_type='cell_line'
)

Column cell_line added to adata.obs
Mapped 1 cell_line ontology terms from `cell_line` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
   input_column input_column_lower    name_lower  ontology_id
0  HEK293T cell       hek293t cell  hek293t cell  CLO:0037372
--------------------------------------------------


### Add disease information

In [30]:
cur_data.create_columns(
    slot='obs',
    col_dict={
        'disease_label': 'healthy',
        'disease_id':None
    },
    overwrite=True
)

# cur_data.standardize_ontology(
#     input_column='disease',
#     column_type='term_name',
#     ontology_type='disease'
# )

Column disease_label added to adata.obs
Column disease_id added to adata.obs


### Add species information

In [31]:
cur_data.create_columns(
    slot="obs",
    col_dict={
        "species": "Homo sapiens"
    }
)

Column species added to adata.obs


### Add sex information

In [32]:
cur_data.create_columns(
    slot="obs",
    col_dict={
        "sex_label": "female",
        "sex_id": None
    }
)

Column sex_label added to adata.obs
Column sex_id added to adata.obs


### Add developmental stage information

In [33]:
cur_data.create_columns(
    slot="obs",
    col_dict={
        "developmental_stage_label": "embryo",
        "developmental_stage_id": None
    }
)

Column developmental_stage_label added to adata.obs
Column developmental_stage_id added to adata.obs


### Match schema column order

In [34]:
cur_data.match_schema_columns(slot='obs')

Matched columns of adata.obs to the obs_schema.


### Validate obs metadata

In [35]:
cur_data.validate_data(slot='obs')

adata.obs is valid according to the obs_schema.
Validated data:


,perturbation_name,perturbed_target_coord,perturbed_target_number,perturbed_target_ensg,perturbed_target_symbol,perturbed_target_biotype,guide_sequence,perturbation_type_label,perturbation_type_id,timepoint,...,cell_type_label,cell_type_id,cell_line_label,cell_line_id,sex_label,sex_id,developmental_stage_label,developmental_stage_id,disease_label,disease_id
0,FAM180A_P1P2-1|FAM180A_P1P2-2,chr7:135728348-135748813;-1,1,ENSG00000189320,FAM180A,protein_coding,GGAGAGCGTCAAATGAAGAG|GGAGGCTGAAGGCTGCGTCC,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None
1,non-targeting_03016|non-targeting_03214,NaN,1,control_nontargeting,control_nontargeting,NaN,GGAGGCTATCTGCTAGACTT|GTCCACATGCGGATAGACAC,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None
2,PCDHGC4_P1P2-1|PCDHGC4_P1P2-2,chr5:141484997-141512975;1,1,ENSG00000242419,PCDHGC4,protein_coding,GGGCCGAACCGCGCCAGAGC|GCGACCGAGAGGCACTGTGT,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None
3,USP30_P1P2-1|USP30_P1P2-2,chr12:109023089-109088023;1,1,ENSG00000135093,USP30,protein_coding,GTGCCGAAGAGGCCGGGACC|GACGACGGTTCCCGAGACAG,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None
4,KLRD1_P1P2-1|KLRD1_P1P2-2,chr12:10226058-10329608;1,1,ENSG00000134539,KLRD1,protein_coding,GATGTTCTTGAATCATCTAT|GGGTTCCTGGAACACTTCAG,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4534294,non-targeting_00183|non-targeting_01885,NaN,1,control_nontargeting,control_nontargeting,NaN,GGGGGAATCTGCAAGACCAT|GAGAAGGAGCGAGGCTAAAG,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None
4534295,AKAP13_P1P2-1|AKAP13_P1P2-2,chr15:85380571-85749358;1,1,ENSG00000170776,AKAP13,protein_coding,GGAGCACAGGCGCTTCACCG|GGGCAGTCTCGGCAGAGCAC,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None
4534296,TTF2_P1P2-1|TTF2_P1P2-2,chr1:117060326-117107453;1,1,ENSG00000116830,TTF2,protein_coding,GAAGTTAGGTGTCCAGAGCA|GAGGTGTCCAGAGCACGGTA,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None
4534297,MMP7_P1P2-1|MMP7_P1P2-2,chr11:102520508-102530750;-1,1,ENSG00000137673,MMP7,protein_coding,GCTCCCGCCTCCTGAGGCAG|GTCCACTGCAATGCTAACAT,CRISPRi,None,P0DT0H0M0S,...,kidney epithelial cell,CL:0002518,HEK293T cell,CLO:0037372,female,None,embryo,None,healthy,None


# VAR slot curation

### Standardise genes

In [36]:
cur_data.show_var()

Variable data:
DataFrame shape: (38606, 1)
--------------------------------------------------
                     gene_symbol
DDX11L2                  DDX11L2
MIR1302-2HG          MIR1302-2HG
FAM138A                  FAM138A
ENSG00000290826  ENSG00000290826
OR4F5                      OR4F5
...                          ...
ENSG00000277836  ENSG00000277836
ENSG00000278633  ENSG00000278633
ENSG00000276017  ENSG00000276017
ENSG00000278817  ENSG00000278817
ENSG00000277196  ENSG00000277196

[38606 rows x 1 columns]
--------------------------------------------------


In [37]:
cur_data.create_columns(slot='var', col_dict={
    'genes':cur_data.adata.var.index
})

Column genes added to adata.var


In [38]:
cur_data.standardize_genes(
    slot="var", input_column="genes", input_column_type="gene_symbol"
)

/content/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:1271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_df["synonyms"] = map_df["synonyms"].str.split("|")


Mapped potential synonyms in genes of the provided dataframe to gene symbols
Converted 36993/38087 gene symbols/ENSG IDs to standardized gene symbols/ENSG IDs
--------------------------------------------------


### Validate var metadata

In [39]:
cur_data.validate_data(slot='var')

adata.var is valid according to the var_schema.
Validated data:


,ensembl_gene_id,gene_symbol
index,,
DDX11L2,ENSG00000236397,DDX11L2
MIR1302-2HG,ENSG00000243485,MIR1302-2HG
FAM138A,ENSG00000237613,FAM138A
ENSG00000290826,ENSG00000290826,None
OR4F5,ENSG00000186092,OR4F5
...,...,...
ENSG00000277836,None,None
ENSG00000278633,None,None
ENSG00000276017,None,None


# Metadata curation

### Auto-populate available metadata

In [40]:
cur_data.populate_exp_metadata()

Experiment metadata populated with available fields from adata.obs:
--------------------------------------------------
{'associated_diseases': [{'term_id': None, 'term_label': 'healthy'}],
 'experiment': {'number_of_perturbed_entities': 4534299,
                'number_of_perturbed_targets': 17790,
                'perturbation_type': [{'term_id': None,
                                       'term_label': 'CRISPRi'}],
                'perturbed_target_biotype': ['protein_coding',
                                             'lncRNA',
                                             'transcribed_unprocessed_pseudogene',
                                             'transcribed_processed_pseudogene',
                                             'Mt_tRNA',
                                             'ribozyme',
                                             'processed_pseudogene',
                                             'unprocessed_pseudogene',
                                           

### Manually curate metadata

Study details

In [41]:
cur_data.add_exp_metadata(
    metadata_slot='study',
    metadata={
        "title": "X-Atlas/Orion: Genome-wide Perturb-seq Datasets via a Scalable Fix-Cryopreserve Platform for Training Dose-Dependent Biological Foundation Models",
        "study_uri": "https://doi.org/10.1101/2025.06.11.659105",
        "year": 2025,
        "first_author": {"first_name": "Ann", "last_name": "Huang"},
        "last_author": {"first_name": "Ci", "last_name": "Chu"},
    }
)

Metadata for 'study' successfully validated:
--------------------------------------------------
{'first_author': {'first_name': 'Ann', 'last_name': 'Huang'},
 'last_author': {'first_name': 'Ci', 'last_name': 'Chu'},
 'study_uri': 'https://doi.org/10.1101/2025.06.11.659105',
 'title': 'X-Atlas/Orion: Genome-wide Perturb-seq Datasets via a Scalable '
          'Fix-Cryopreserve Platform for Training Dose-Dependent Biological '
          'Foundation Models',
 'year': 2025}
--------------------------------------------------


Experiment details

In [42]:
cur_data.add_exp_metadata(
    metadata_slot='experiment',
    metadata={
        "title": "Over 4.5 million HEK293T cells were transfected with dual gRNAs targeting the same gene, and whole genome CRISPRi Perturb-seq was performed.",
        "summary": "The researchers developed the Fix-Cryopreserve-ScRNAseq (FiCS) Perturb-seq platform to enable scalable and reproducible genome-wide perturbation screening in human cell lines. Using this platform, they generated a genome-wide dual-guide CRISPRi Perturb-seq dataset in HCT116 colon carcinoma cells. The dataset includes over 3.4 million deeply sequenced cells, with each cell yielding over 16,000 unique molecular identifiers (UMIs).",
        "replicates": "technical",
        "number_of_samples": 223
    }
)

Metadata for 'experiment' successfully validated:
--------------------------------------------------
{'number_of_perturbed_entities': 4534299,
 'number_of_perturbed_targets': 17790,
 'number_of_samples': 223,
 'perturbation_type': [{'term_id': None, 'term_label': 'CRISPRi'}],
 'perturbed_target_biotype': ['protein_coding',
                              'lncRNA',
                              'transcribed_unprocessed_pseudogene',
                              'transcribed_processed_pseudogene',
                              'Mt_tRNA',
                              'ribozyme',
                              'processed_pseudogene',
                              'unprocessed_pseudogene',
                              'TR_C_gene',
                              'transcribed_unitary_pseudogene'],
 'perturbed_targets': ['ENSG00000189320',
                       'control_nontargeting',
                       'ENSG00000242419',
                       'ENSG00000135093',
                       'ENS

Perturbation details

In [43]:
cur_data.add_exp_metadata(
    metadata_slot='perturbation',
    metadata={
         "library_generation_type": {
            "term_id": "EFO:0022868",
            "term_label": "endogenous",
        },
        "library_generation_method": {
            "term_id": "EFO:0022895",
            "term_label": "dCas9-KRAB",
        },
        "enzyme_delivery_method": {
            "term_id": None,
            "term_label": "nanoparticle-mediated transfection",
        },
        "library_delivery_method": {
            "term_id": None,
            "term_label": "lentiviral transduction",
        },
        "enzyme_integration_state": {
            "term_id": None,
            "term_label": "random locus integration",
        },
        "library_integration_state": {
            "term_id": None,
            "term_label": "random locus integration",
        },
        "enzyme_expression_control": {
            "term_id": None,
            "term_label": "constitutive expression",
        },
        "library_expression_control": {
            "term_id": None,
            "term_label": "constitutive expression",
        },
        "library": {
            "library_name": "custom",
            "accession": None,
            "library_format": {
                "term_id": None,
                "term_label": "pooled",
            },
            "library_scope": {
                "term_id": None,
                "term_label": "genome-wide",
            },
            "library_perturbation_type": [
                {
                    "term_id": None,
                    "term_label": "inhibition",
                },
            ],
            "manufacturer": "Weissman",
            "lentiviral_generation": "3",
            "grnas_per_gene": "2",
            "total_grnas": "20845",
            "total_variants": None
        }
    }
)

Metadata for 'perturbation' successfully validated:
--------------------------------------------------
{'enzyme_delivery_method': {'term_id': None,
                            'term_label': 'nanoparticle-mediated transfection'},
 'enzyme_expression_control': {'term_id': None,
                               'term_label': 'constitutive expression'},
 'enzyme_integration_state': {'term_id': None,
                              'term_label': 'random locus integration'},
 'library': {'accession': None,
             'grnas_per_target': None,
             'lentiviral_generation': '3',
             'library_format': {'term_id': None, 'term_label': 'pooled'},
             'library_name': 'custom',
             'library_perturbation_type': [{'term_id': None,
                                            'term_label': 'inhibition'}],
             'library_scope': {'term_id': None, 'term_label': 'genome-wide'},
             'manufacturer': 'Weissman',
             'total_grnas': '20845',
            

Assay details

In [44]:
cur_data.add_exp_metadata(
    metadata_slot='assay',
    metadata={
        "readout_dimensionality": {
            "term_id": None,
            "term_label": "high-dimensional assay",
        },
        "readout_type": {
            "term_id": None,
            "term_label": "transcriptomic",
        },
        "readout_technology": {
            "term_id": None,
            "term_label": "single-cell rna-seq",
        },
        "method_name": {
            "term_id": None,
            "term_label": "Perturb-seq",
        },
        "method_uri": None,
        "sequencing_library_kit": {
            "term_id": None,
            "term_label": "10x Genomics Chromium GEM-X Single Cell 5-prime kit v3",
        },
        "sequencing_platform": {"term_id": None, "term_label": "Illumina NovaSeq X Plus"},
        "sequencing_strategy": {"term_id": None, "term_label": "barcode sequencing"},
        "software_counts": {"term_id": None, "term_label": "CellRanger"},
        "software_analysis": {"term_id": None, "term_label": "custom"},
        "reference_genome": {
            "term_id": None,
            "term_label": "GRCh38",
        }
    }
)

Metadata for 'assay' successfully validated:
--------------------------------------------------
{'method_name': {'term_id': None, 'term_label': 'Perturb-seq'},
 'method_uri': None,
 'readout_dimensionality': {'term_id': None,
                            'term_label': 'high-dimensional assay'},
 'readout_technology': {'term_id': None, 'term_label': 'single-cell rna-seq'},
 'readout_type': {'term_id': None, 'term_label': 'transcriptomic'},
 'reference_genome': {'term_id': None, 'term_label': 'GRCh38'},
 'sequencing_library_kit': {'term_id': None,
                            'term_label': '10x Genomics Chromium GEM-X Single '
                                          'Cell 5-prime kit v3'},
 'sequencing_platform': {'term_id': None,
                         'term_label': 'Illumina NovaSeq X Plus'},
 'sequencing_strategy': {'term_id': None, 'term_label': 'barcode sequencing'},
 'software_analysis': {'term_id': None, 'term_label': 'custom'},
 'software_counts': {'term_id': None, 'term_label'

Model system details

In [45]:
cur_data.add_exp_metadata(
    metadata_slot='model_system',
    metadata={
        "species": "Homo sapiens",
        "passage_number": None,
        }
)

Metadata for 'model_system' successfully validated:
--------------------------------------------------
{'cell_line': [{'term_id': 'CLO:0037372', 'term_label': 'HEK293T cell'}],
 'cell_type': [{'term_id': 'CL:0002518',
                'term_label': 'kidney epithelial cell'}],
 'developmental_stage': [{'term_id': None, 'term_label': 'embryo'}],
 'model_system': [{'term_id': None, 'term_label': 'cell line'}],
 'passage_number': None,
 'sex': [{'term_id': None, 'term_label': 'female'}],
 'species': 'Homo sapiens',
 'tissue': [{'term_id': 'UBERON:0002113', 'term_label': 'kidney'}]}
--------------------------------------------------


Associated dataset details

In [46]:
cur_data.add_exp_metadata(
    metadata_slot='associated_datasets',
    metadata=[
        {
            "dataset_accession": "HEK293T_filtered_dual_guide_cells",
            "dataset_uri": "https://plus.figshare.com/ndownloader/files/55074802",
            "dataset_description": "Filtered raw counts",
            "dataset_file_name": "HEK293T_filtered_dual_guide_cells.h5ad",
        }
    ]
)

Metadata for 'associated_datasets' successfully validated:
--------------------------------------------------
[{'dataset_accession': 'HEK293T_filtered_dual_guide_cells',
  'dataset_description': 'Filtered raw counts',
  'dataset_file_name': 'HEK293T_filtered_dual_guide_cells.h5ad',
  'dataset_uri': 'https://plus.figshare.com/ndownloader/files/55074802'}]
--------------------------------------------------


### Validate metadata

In [47]:
cur_data.validate_exp_metadata()

Experiment metadata successfully validated:
--------------------------------------------------
{'assay': {'method_name': {'term_id': None, 'term_label': 'Perturb-seq'},
           'method_uri': None,
           'readout_dimensionality': {'term_id': None,
                                      'term_label': 'high-dimensional assay'},
           'readout_technology': {'term_id': None,
                                  'term_label': 'single-cell rna-seq'},
           'readout_type': {'term_id': None, 'term_label': 'transcriptomic'},
           'reference_genome': {'term_id': None, 'term_label': 'GRCh38'},
           'sequencing_library_kit': {'term_id': None,
                                      'term_label': '10x Genomics Chromium '
                                                    'GEM-X Single Cell 5-prime '
                                                    'kit v3'},
           'sequencing_platform': {'term_id': None,
                                   'term_label': 'Illumina Nova

# Save the dataset

In [48]:
cur_data.save_curated_data()

Curated data saved to Perturbseq/curated/h5ad/orion_2025_hek293t_curated.h5ad


In [49]:
!gcloud storage cp Perturbseq/curated/h5ad/orion_2025_hek293t_curated.h5ad gs://perturbation-catalogue-lake/perturbseq/curated/

uploading large objects. If you would like to opt-out and instead
perform a normal upload, run:
`gcloud config set storage/parallel_composite_upload_enabled False`
If you would like to disable this warning, run:
`gcloud config set storage/parallel_composite_upload_enabled True`
Note that with parallel composite uploads, your object might be
uploaded as a composite object
(https://cloud.google.com/storage/docs/composite-objects), which means
that any user who downloads your object will need to use crc32c
checksums to verify data integrity. gcloud storage is capable of
computing crc32c checksums, but this might pose a problem for other
clients.

Copying file://Perturbseq/curated/h5ad/orion_2025_hek293t_curated.h5ad to gs://perturbation-catalogue-lake/perturbseq/curated/orion_2025_hek293t_curated.h5ad

Average throughput: 524.1MiB/s
